In [2]:
import pandas as pd
import numpy as np
import keras
import jieba
import re

Using TensorFlow backend.


In [3]:
#读取数据
train_data = pd.read_csv('dp_utf8.csv', error_bad_lines=False,lineterminator='\n',skipinitialspace=True)
train_data

/Users/anthonymao08/anaconda3/envs/tensorflow/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Review_ID,Merchant,Rating,Score_taste,Score_environment,Score_service,Price_per_person,Time,Num_thumbs_up,Num_ response,Content_review,Reviewer,Reviewer_value,Reviewer_rank,Favorite_foods
0,377313283,炳胜品味(珠江新城店),3,3,3,2,空,10月23日,2,1,来广州出差，住在附近的康莱德酒店，晚上开完会就慕名而来，大概七点四十分的样子到的，人还有很多...,胖子界的瘦子吴,5,1,\r
1,376404966,炳胜品味(珠江新城店),4,3,4,3,空,10月19日,2,0,?·??·??前言本来是朋友约我们来的，结果朋友临时有事走不开，就我俩在这儿吃了。?·??·...,快快到盆里来,6,1,\r
2,374370927,炳胜品味(珠江新城店),4,3,3,2,空,10月9日,0,0,朋友结婚选择在这里摆酒，在猎德A出口出来，直行转入兴盛路？再直行，走到天汇广场，过马路，再一...,Yyg喵,3,1,\r
3,377270662,炳胜品味(珠江新城店),4,3,3,3,155,10月23日,0,0,据说是个老子号，听名字就感觉很广东??，很正宗的样子！！这家分店打听了一下，说是地儿最大，环...,肥螳螂_,4,1,干炒牛河 炳胜荔枝鸡 \r
4,377184095,炳胜品味(珠江新城店),4,3,3,3,空,10月23日,0,0,周末不煮饭的日子，偶来啦??。中午12点多到店，3楼满了??让到2楼，2楼说要到1楼拿号??...,hhy_0919,4,1,\r
5,376654771,炳胜品味(珠江新城店),4,3,4,3,空,10月20日,1,0,环境特别优雅，一进门让人感觉特别舒服，价位适中。推荐几道菜#瑶柱佛手瓜煲龙#汤喝起来特别鲜甜...,噜啦啦璐璐,5,1,\r
6,371978056,炳胜品味(珠江新城店),5,4,4,3,空,9月28日,8,0,晚上看小蛮腰，坐公交坐到这块，然后搜了一下想说那晚饭就他家吧。大厅还算比较宽敞，晚上好像一直...,哆啦瞄瞄,7,1,\r
7,372562660,炳胜品味(珠江新城店),4,3,4,4,空,10月1日,2,0,交通便利，昨天是放假前一天，不敢开车去，所以选择了地铁，在APM剧院站上来步行10分钟就到了...,Henry莫,6,1,\r
8,373441960,炳胜品味(珠江新城店),4,3,3,3,空,10月5日,5,0,晚上差不多七点半到的那里，要排队，表示生意还真的挺好的，推测味道也应该挺好的。排了差不多半小...,趟七,5,1,脆皮叉烧拼秘制黑叉烧 豉油皇鹅肠 \r
9,375713033,炳胜品味(珠江新城店),3,1,4,2,90,10月15日,0,0,先说环境不错！不会想其他分店桌与桌之间靠得很近！这间位置很宽松，坐得也听舒服，不过灯光有点暗...,JMEAAKN,4,1,招牌菠萝包 脆皮叉烧拼秘制黑叉烧 古法烧鹅 \r


In [4]:
train_data=train_data.drop(['Merchant','Review_ID','Score_taste','Score_environment','Score_service','Price_per_person','Time','Reviewer','Reviewer_value','Reviewer_rank','Num_thumbs_up','Num_ response'],axis=1)

In [5]:
#data_column=list(train_data.columns)
train_data = train_data.drop('Favorite_foods\r',axis=1)

In [6]:
train_data['Rating'].describe()

count    467455.000000
mean          4.216292
std           0.962893
min           1.000000
25%           4.000000
50%           4.000000
75%           5.000000
max           5.000000
Name: Rating, dtype: float64

In [17]:
sample = train_data.sample(n=50000, replace=False, axis=0)
sample.describe()
dataset_1 = train_data[train_data['Rating'] == 1].sample(n=10000, replace=False, axis=0)
dataset_2 = train_data[train_data['Rating'] == 2].sample(n=10000, replace=False, axis=0)
dataset_3 = train_data[train_data['Rating'] == 3].sample(n=10000, replace=False, axis=0)
dataset_4 = sample[sample['Rating'] == 4]
dataset_4 = dataset_4.sample(n=10000, replace=False, axis=0)
dataset_5 = sample[sample['Rating'] == 5]
dataset_5 = dataset_5.sample(n=10000, replace=False, axis=0)

In [19]:
dataset=pd.concat([dataset_1, dataset_2, dataset_3, dataset_4, dataset_5])
dataset.describe()

,Rating
count,50000.000000
mean,3.000000
std,1.414228
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,5.000000


In [24]:
dataset['Content_review'] = dataset['Content_review'].astype(str)
for i in dataset['Content_review']:
    i = jieba.cut(i, cut_all = True)
dataset['Content_review'] = dataset['Content_review'].apply(lambda x:' '.join(jieba.cut(x)))
dataset

,Rating,Content_review
137055,1,去过 这么 多家 酒楼 这家 最差 ， 餐具 茶水 很久 才 到 ， 好不容易 等到 点 完...
131838,1,很 难吃 ， 煮 鱼 不放 姜 ， 不 值得 推荐 差
22091,1,服务态度 严重 的 差评 ， 表示 以后 不会 去 了 。 气 的 什么 吃饭 心情 的 都...
406633,1,2017.9 . 16 号 22 : 30 分 ， 一个 穿 灰色 衣服 戴着 对讲机 ， ...
326628,1,这家 店 简直 太 让 人 气愤 了 ， 没有 去过 的 就 不必 去 了 ！ 2 月 27...
187083,1,服务 非常 差 ， 菜量 很少 ， 不值 菜价 ， 服务员 还 很 理所当然 ， 我 不是 ...
124713,1,功夫茶 A11 房间 ， 空调 直吹 ， 冷到 盖 上 毛毯 穿 上 外套 都 觉得 冷 ，...
328263,1,超级 不 满意 ， 喝 早茶 价格 超级 贵 ， 差点 太坑 了
147122,1,服务 爰 理 不理 ， 难以 接受 6 道 热菜 中 全变 冷盘 ， 只有 一道 是 微温 ，
392052,1,服务 极差 ， 停车场 保安 态度 嚣张 ， 房间 手机 全无 联通 信号 ！ 以后 不会 ...


In [25]:
#提取特征
'''from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=500)
tokenizer.fit_on_texts(dataset['Content_review'])
list_tokenized_train = tokenizer.texts_to_sequences(dataset['Content_review'])
x = pad_sequences(list_tokenized_train, maxlen=120)
y = dataset['Rating']'''

MAX_SEQUENCE_LENGTH = 120 # 每条新闻最大长度
EMBEDDING_DIM = 200 # 词向量空间维度
VALIDATION_SPLIT = 0.16 # 验证集比例
TEST_SPLIT = 0.2 # 测试集比例

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

tokenizer = Tokenizer()
tokenizer.fit_on_texts(dataset['Content_review'])
sequences = tokenizer.texts_to_sequences(dataset['Content_review'])
word_index = tokenizer.word_index
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(dataset['Rating']))

In [26]:
#划分数据集
from sklearn.model_selection import train_test_split
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

p1 = int(len(data)*(1-VALIDATION_SPLIT-TEST_SPLIT))
p2 = int(len(data)*(1-TEST_SPLIT))
x_train = data[:p1]
y_train = labels[:p1]
x_val = data[p1:p2]
y_val = labels[p1:p2]
x_test = data[p2:]
y_test = labels[p2:]

In [163]:
'''from keras.utils import np_utils
y_train_onehot = np_utils.to_categorical(y_train)
y_test_onehot = np_utils.to_categorical(y_test)'''

'from keras.utils import np_utils\ny_train_onehot = np_utils.to_categorical(y_train)\ny_test_onehot = np_utils.to_categorical(y_test)'

In [27]:
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Sequential

model = Sequential()
model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(Dropout(0.2))
model.add(Conv1D(250, 3, padding='valid', activation='relu', strides=1))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(EMBEDDING_DIM, activation='relu'))
model.add(Dense(labels.shape[1], activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 200)          14913200  
_________________________________________________________________
dropout_1 (Dropout)          (None, 120, 200)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 118, 250)          150250    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 39, 250)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9750)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               1950200   
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 1206      
Total para

In [165]:
'''模型搭建
from keras.models import Model, Sequential
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten, Bidirectional, GlobalMaxPool1D
embed_size = 128
max_features = 1000
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])'''

'模型搭建\nfrom keras.models import Model, Sequential\nfrom keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten, Bidirectional, GlobalMaxPool1D\nembed_size = 128\nmax_features = 1000\nmodel = Sequential()\nmodel.add(Embedding(max_features, embed_size))\nmodel.add(GlobalMaxPool1D())\nmodel.add(Dense(20, activation="relu"))\nmodel.add(Dropout(0.05))\nmodel.add(Dense(1,activation="sigmoid"))\nmodel.compile(loss=\'binary_crossentropy\', optimizer=\'adam\', metrics=[\'accuracy\'])'

In [28]:
#训练模型
model.fit(x_train, y_train, batch_size=500, epochs=5, validation_split=0.2)

Train on 25599 samples, validate on 6400 samples
Epoch 1/5
25599/25599 [==============================] - 121s 5ms/step - loss: 0.2991 - acc: 0.8415 - val_loss: 0.6655 - val_acc: 0.8501
Epoch 2/5
25599/25599 [==============================] - 119s 5ms/step - loss: 0.2397 - acc: 0.8800 - val_loss: 0.7048 - val_acc: 0.8257
Epoch 3/5
25599/25599 [==============================] - 118s 5ms/step - loss: 0.1861 - acc: 0.9155 - val_loss: 0.8574 - val_acc: 0.7910
Epoch 4/5
25599/25599 [==============================] - 118s 5ms/step - loss: 0.1253 - acc: 0.9487 - val_loss: 0.8691 - val_acc: 0.8118
Epoch 5/5
25599/25599 [==============================] - 119s 5ms/step - loss: 0.0784 - acc: 0.9703 - val_loss: 0.9814 - val_acc: 0.8010


In [29]:
accuracy = model.evaluate(x_test, y_test, batch_size = 50)
print("test accuracy:{}".format(accuracy[1]))

10000/10000 [==============================] - 15s 1ms/step 
test accuracy:0.6857666024565696
